<a href="https://colab.research.google.com/github/AnshikaaAgarwal/handgesturerecog_raspberrypi/blob/main/Hand_Gesture_Recog_for_RaspberryPi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import os


In [3]:
import zipfile

# Define the uploaded file name
dataset_zip = "/content/archive (6).zip"

# Extract the dataset
with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall("sign_language_digits")

print("Dataset extracted successfully!")


Dataset extracted successfully!


In [51]:
import numpy as np

# Load the dataset
X = np.load('/content/sign_language_digits/X.npy')  # Features (images)
Y = np.load('/content/sign_language_digits/Y.npy')  # Labels (digit classes)

# Check the shapes
print(f"X shape: {X.shape}")
print(f"Y shape: {Y.shape}")


X shape: (2062, 64, 64)
Y shape: (2062, 10)


In [52]:
# Step 1: Normalize the pixel values to [0, 1]
X = X.astype('float32') / 255.0

# Check the reshaped data
print(f"Reshaped X shape: {X.shape}")

# Step 2: Since images are grayscale, we need to ensure the shape is (64, 64, 1) for each image
# This step ensures that the model will expect 1 channel for grayscale images
if X.shape[-1] != 1:
    X = np.expand_dims(X, axis=-1)  # Add a channel dimension if not already present

# Step 3: Split data into training and testing sets (80% train, 20% test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



# Verify the shape
print(f"y_train data shape: {Y_train.shape}")
print(f"y_test data shape: {Y_test.shape}")






Reshaped X shape: (2062, 64, 64)
y_train data shape: (1649, 10)
y_test data shape: (413, 10)


In [64]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Improved model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128),
    Dropout(0.5),  # Regularization layer to prevent overfitting
    Dense(64, activation=LeakyReLU(alpha=0.2)),  # Leaky ReLU for better performance
    Dropout(0.5),  # Dropout layer
    Dense(10, activation='softmax')  # 10 output classes (digits 0-9)
])

# Compile the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary to check the architecture
model.summary()

# Data augmentation setup
datagen = ImageDataGenerator(
    rotation_range=15,  # Random rotations
    width_shift_range=0.1,  # Random horizontal shifts
    height_shift_range=0.1,  # Random vertical shifts
    shear_range=0.1,  # Random shearing
    zoom_range=0.1,  # Random zoom
    horizontal_flip=True,  # Random horizontal flips
    fill_mode='nearest'  # Filling in missing pixels after transformations
)

# Fit the model with data augmentation
datagen.fit(X_train)

# Train the model
model.fit(datagen.flow(X_train, Y_train, batch_size=32), epochs=50, validation_data=(X_test, Y_test))



Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_33 (Conv2D)                   │ (None, 62, 62, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 62, 62, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_33 (MaxPooling2D)      │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_34 (Conv2D)                   │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 29, 29, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_34 (MaxPooling2D)      │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_35 (Conv2D)                   │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 12, 12, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_35 (MaxPooling2D)      │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_14 (Flatten)                 │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 128)                 │         589,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 692,426 (2.64 MB)

 Trainable params: 691,978 (2.64 MB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 10s 83ms/step - accuracy: 0.1338 - loss: 3.5617 - val_accuracy: 0.0775 - val_loss: 2.3213
Epoch 2/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.2908 - loss: 2.0218 - val_accuracy: 0.0775 - val_loss: 2.3439
Epoch 3/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.3570 - loss: 1.7249 - val_accuracy: 0.0775 - val_loss: 2.3688
Epoch 4/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.4319 - loss: 1.5966 - val_accuracy: 0.0847 - val_loss: 2.3601
Epoch 5/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.5035 - loss: 1.4584 - val_accuracy: 0.0847 - val_loss: 2.3577
Epoch 6/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5296 - loss: 1.3390 - val_accuracy: 0.0847 - val_loss: 2.3906
Epoch 7/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.5655 - loss: 1.2184 - val_accuracy: 0.0993 - val_loss: 2.4053
Epoch 8/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5906 - loss: 1.1605 - val_accuracy: 0.0993 - 

In [65]:
# Step 1: Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9415 - loss: 0.1798 
Test Loss: 0.21572953462600708
Test Accuracy: 0.9273607730865479


In [68]:
import tensorflow as tf

# Convert the trained model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted TFLite model
with open('gesture_recognition_model.tflite', 'wb') as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpjmc_0v4k'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 1), dtype=tf.float32, name='keras_tensor_119')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  132761956027728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132761956037232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132761956033888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132761956028960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132761956039696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132761954112560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132761954113440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132761954113264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132761954111152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132761954113968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13276195410